## Tentative de scrapper le site "https://www.trackingthepros.com"

In [3]:
import requests
from bs4 import BeautifulSoup

def get_player_name_by_id(player_id):
    url = f"https://www.trackingthepros.com/s/player/{player_id}/"
     
    # Ajout d'un en-tête d'utilisateur pour simuler une requête de navigateur web
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        player_name_tag = soup.find("h2", class_="panel-title")
        
        if player_name_tag:
            player_name = player_name_tag.text.strip()
            return player_name
        else:
            return None
    else:
        print(f"Échec de la requête pour l'ID {player_id}. Code de statut :", response.status_code)
        return None

def main():
    # Liste des IDs que vous souhaitez extraire les noms de joueur
    ids = ["2195", "585"]  # Remplacez par les IDs que vous souhaitez
    
    for player_id in ids:
        player_name = get_player_name_by_id(player_id)
        if player_name:
            print(f"ID : {player_id}, Nom du joueur : {player_name}")
        else:
            print(f"ID : {player_id}, Joueur non trouvé.")

if __name__ == "__main__":
    main()

ID : 2195, Joueur non trouvé.
ID : 585, Joueur non trouvé.


# Scrapping pour obtenir une liste de summoner name

## Defintion des fonctions

In [4]:
import requests
from bs4 import BeautifulSoup
import re 

def scrapping_summoner_name(name):

    # Remplacez l'URL ci-dessous par l'adresse du site web que vous souhaitez scraper
    url = f"https://lol.fandom.com/wiki/{name}"

    # Faites une requête pour obtenir le contenu HTML de la page
    response = requests.get(url)

    # Vérifiez si la requête s'est bien passée
    if response.status_code == 200:
        # Analysez le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Recherchez la balise "<td>" avec la classe "infobox-label" et le texte "Soloqueue IDs"
        target_td = soup.find("td", class_="infobox-label", string="Soloqueue IDs")
        # Vérifiez si la balise a été trouvée
        if target_td:
            # Utilisez la méthode .find_next_sibling() pour accéder au <td> suivant
            next_td = target_td.find_next_sibling("td")
            # Vérifiez si le <td> suivant a été trouvé
            if next_td:
                summoner_dict = {}
                # Utilisez la méthode .find_all() pour rechercher toutes les balises <b> dans le <td> suivant
                if next_td.find_all("b") != []:
                    sub_b_tags = next_td.find_all("b")
                    # Parcourez les balises <b> pour obtenir les informations par région
                    current_region = None
                    for b_tag in sub_b_tags:
                        region_name = b_tag.text.strip()
                        if str(b_tag.next_sibling) != "<br/>":
                            region_info = b_tag.next_sibling.strip()
                            if region_name.endswith(":"):
                                # Si la balise <b> termine par ":" cela signifie que c'est une nouvelle région
                                current_region = region_name[:-1]  # Supprimer le ":" à la fin du nom de région
                                summoner_dict[current_region] = region_info    
                            else :
                                current_region = region_name[:]
                                summoner_dict[current_region] = region_info[2:]
                else:
                    def extract_summoner_names(input_string):
                        # Recherche de la première parenthèse
                        start_index = input_string.find('(')
                        
                        # Si pas de parenthèse, retourner un dictionnaire vide
                        if start_index == -1:
                            return {}

                        # Recherche de la dernière parenthèse
                        end_index = input_string.find(')')

                        # Si une seule paire de parenthèses est trouvée
                        if input_string.count('(') == 1 and input_string.count(')') == 1:
                            region = input_string[start_index+1:end_index]
                            summoner_name = input_string[:start_index].strip()
                            return {region: summoner_name}

                        # Sinon, plusieurs paires de parenthèses sont trouvées
                        regions = [part.strip() for part in input_string[start_index+1:end_index].split(',')]
                        summoner_name = input_string[:start_index].strip()

                        # Vérification de la correspondance entre le nombre de régions et de noms d'invocateur
                        if len(regions) == 1:
                            return {regions[0]: summoner_name}
                        elif len(regions) == len(summoner_name.split(',')):
                            return dict(zip(regions, summoner_name.split(',')))
                        else:
                            return {}

            else:
                print("Le <td> suivant n'a pas été trouvé.")
                return {}
        else:
            print("Le <td> avec le texte 'Soloqueue IDs' n'a pas été trouvé.")
            return {}
    else:
        print("La requête n'a pas abouti. Code de statut :", response.status_code)
        return {}

    return summoner_dict

import re

def scrapping_wiki_lol(url):
    # Get the content of the page
    response = requests.get(url)
    html_content = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all the <b> tags within <td> tags
    player_names = soup.select('td b')

    liste_names = []
    # Extract and print the names of the players, filtering out unwanted data
    for name in player_names:
        player_name = name.text.strip()
        if re.search(r'\d', player_name) is None:  # Check if the name contains any digits
            liste_names.append(player_name)
    return liste_names

## LFL

In [5]:
url = "https://fr.wikipedia.org/wiki/Ligue_fran%C3%A7aise_de_League_of_Legends"
liste_names_lfl = scrapping_wiki_lol(url)

print(liste_names_lfl)

# Create an empty dictionary to hold all the data
dico_summoner_names_lfl= {}

for name in liste_names_lfl :
    print(name)
    dico = scrapping_summoner_name(name)
    dico_summoner_names_lfl[name] = dico


print(f"Le dico contenant le nom des joueurs et leur summoner_name est : {dico_summoner_names_lfl}")


['Ragner', 'Lyncas', 'toucouille', 'Jezu', 'kamilius', 'Duke', 'Badlulu', 'Eckas', 'twohoyrz', 'Practice', 'Raxxo', 'Emi', 'Fara', 'Cabochard', 'Cinkrof', 'Saken', 'Caliste', 'Targamas', 'Reha', 'Skeanz', 'Wadi', 'Kryze', 'White', 'Backlund', 'Jeskla', 'Zoelys', 'Hansen', 'Hairost', 'Kio', 'Djoko', 'Nomanz', 'Rayito', 'Klaj', 'AsylumKubz', 'Manaty', 'Rafa', 'Howling', 'Pridestalkr', 'Czekolad', 'Trigger', 'Veignorem', 'Jarge', 'Melonik', 'Shlatan', 'Peng', 'TakeSet', 'Steeelback', 'KiKi', 'Agresivoo', 'MAXI', 'Reeker', 'xMatty', 'Erdote', 'delord', 'Lot', 'SkewMond', 'Eika', 'Shiganari', 'Hantera', 'xani', 'Scarface', 'Daglas', 'Czajek', 'Neon', 'Jactroll', 'Nahovsky', 'Patkica', 'Innaxe', 'Krog', 'Chips', 'Tweekz', 'Noi', 'Glopo', 'Trayton', 'Marex', 'Rhobalas', 'Chreak', 'Bulii', 'Printemps', 'Été', 'Finale']
Ragner
Lyncas
toucouille
Jezu
kamilius
Duke
Le <td> avec le texte 'Soloqueue IDs' n'a pas été trouvé.
Badlulu
Le <td> avec le texte 'Soloqueue IDs' n'a pas été trouvé.
Eckas
two

In [6]:
url = "https://fr.wikipedia.org/wiki/League_of_Legends_EMEA_Championship"
liste_names_emea = scrapping_wiki_lol(url)

print(liste_names_emea)

# Create an empty dictionary to hold all the data
dico_summoner_names_emea= {}

for name in liste_names_emea :
    print(name)
    dico = scrapping_summoner_name(name)
    dico_summoner_names_emea[name] = dico


print(f"Le dico contenant le nom des joueurs et leur summoner_name est : {dico_summoner_names_emea}")


['Finn', 'LIDER', 'Kobbe', 'JeongHoon', 'AoD', 'Jandro', 'Odoamne', 'Peach', 'Abbedagge', 'Patrik', 'LIMIT', 'Hidon', 'Kaas', 'Oscarinin', 'Razork', 'Humanoid', 'Noah', 'Trymbi', 'Nightshare', 'Rekkles', 'Hiiva', 'BrokenBlade', 'Yike', 'Caps', 'Hans Sama', 'Mikyx', 'Dylan Falco', 'Szygenda', 'Malrang', 'Larssen', 'Comp', 'Advienne', 'Arvindir', 'Chasy', 'Elyoya', 'Nisqy', 'Carzzy', 'Hylissang', 'Mac', 'Pad', 'Irrelevant', 'Markoon', 'Sertuss', 'Exakick', 'Doss', 'Swiffer', 'Adam', 'Sheo', 'nuc', 'Crownie', 'Labrov', 'GotoOne', 'Striker', 'Evi', 'Jankos', 'Vetheo', 'Flakked', 'Mersa', 'Peter Dun', 'SeeEl', 'Photon', 'Daglas', 'Perkz', 'Upset', 'Kaiser', 'Carter', 'Bo', 'Hjarnan', 'Classement', 'Équipes', 'Résultats', 'Classement', 'Équipes', 'Dotations/Conséquences', 'Classement', 'Équipes', 'Résultats', 'Classement', 'Équipes', 'Dotations/Conséquences', '[*]', 'Classement', 'Équipes', 'Résultats', 'Classement', 'Équipes', 'Dotations/Conséquences', 'Classement', 'Équipes', 'Résultats', 

In [10]:
url = "https://fr.wikipedia.org/wiki/League_of_Legends_Champions_Korea"
liste_names_lck = scrapping_wiki_lol(url)

print(liste_names_lck)

# Create an empty dictionary to hold all the data
dico_summoner_names_lck= {}

for name in liste_names_lck :
    print(name)
    dico = scrapping_summoner_name(name)
    dico_summoner_names_lck[name] = dico


print(f"Le dico contenant le nom des joueurs et leur summoner_name est : {dico_summoner_names_lck}")

['Doran', 'Peanut', 'Chovy', 'Peyz', 'Delight', 'Score', 'Museong', 'Mafa', 'Zeus', 'Oner', 'Faker', 'Gumayusi', 'Keria', 'Bengi', 'Sky', 'Roach', 'Burdol', 'Willer', 'Clozer', 'Teddy', 'Kael', 'Ryu', 'Lyn', 'Canna', 'Canyon', 'ShowMaker', 'Deft', 'Kellin', 'Acorn', 'GorillA', 'Kiin', 'Cuzz', 'Bdd', 'Aiming', 'Lehends', 'Hirai', 'supreme', 'Rascal', 'Croco', 'FATE', 'Paduck', 'BeryL', 'Micro', 'Juhan', 'kyeahoo', 'Shine', 'DuDu', 'YoungJae', 'BuLLDoG', 'Taeyoon', 'Jun', 'cvMax', 'Moham', 'Alvingo', 'DnDn', 'Sylvie', 'FIESTA', 'vital', 'Peter', 'Irean', 'Quad', 'Coco', 'Morgan', 'UmTi', 'Karis', 'Hena', 'Effort', 'Edgar', 'Drinker', 'Kingen', 'Clid', 'ZEKA', 'Viper', 'Life', 'DanDy', 'Mowgli']
Doran
Le <td> avec le texte 'Soloqueue IDs' n'a pas été trouvé.
Peanut
Chovy
Peyz
Delight
Score
Museong
Mafa
Zeus
Oner
Faker
Gumayusi
Keria
Bengi
Sky
Le <td> avec le texte 'Soloqueue IDs' n'a pas été trouvé.
Roach
Burdol
Willer
Clozer
Teddy
Kael
Le <td> avec le texte 'Soloqueue IDs' n'a pas été tr

In [15]:
import pickle
import os 

# Chemin absolu du dossier courant
chemin_scouting = os.path.abspath('..')

print("Chemin du dossier parent :", chemin_scouting)

chemin_data = chemin_scouting + "/datas/summoner_names/"

with open(chemin_data + "dico_summoner_names_lfl.pkl", 'wb') as f:
    pickle.dump(dico_summoner_names_lck, f)

with open(chemin_data + 'dico_summoner_names_emea.pkl', 'wb') as f:
    pickle.dump(dico_summoner_names_lck, f)

with open(chemin_data + 'dico_summoner_names_lck.pkl', 'wb') as f:
    pickle.dump(dico_summoner_names_lck, f)

Chemin du dossier parent : /Users/atysp/Desktop/Avisia/projet_lol/scouting


## option à modifier

In [ ]:
def extract_summoner_names(input_string):
    # Recherche de la première parenthèse
    start_index = input_string.find('(')
    
    # Si pas de parenthèse, retourner un dictionnaire vide
    if start_index == -1:
        return {}

    # Recherche de la dernière parenthèse
    end_index = input_string.find(')')

    # Si une seule paire de parenthèses est trouvée
    if input_string.count('(') == 1 and input_string.count(')') == 1:
        region = input_string[start_index+1:end_index]
        summoner_name = input_string[:start_index].strip()
        return {region: summoner_name}

    # Sinon, plusieurs paires de parenthèses sont trouvées
    regions = [part.strip() for part in input_string[start_index+1:end_index].split(',')]
    summoner_name = input_string[:start_index].strip()

    # Vérification de la correspondance entre le nombre de régions et de noms d'invocateur
    if len(regions) == 1:
        return {regions[0]: summoner_name}
    elif len(regions) == len(summoner_name.split(',')):
        return dict(zip(regions, summoner_name.split(',')))
    else:
        return {}

# Exemples d'utilisation :
print(extract_summoner_names("KC Gaulois (EUW), otwoeur11 (KR)"))
print(extract_summoner_names("J'ohn Doe (NA)"))
print(extract_summoner_names("Djoko (EUNE), Elile"))
print(extract_summoner_names("KC Gaulois , otwoeur11"))
print(extract_summoner_names("KC Gaulois"))


{'EUW': 'KC Gaulois'}
{'NA': "J'ohn Doe"}
{'EUNE': 'Djoko'}
{}
{}
